In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [5]:

tagged_orignal_df = pd.read_csv('../data/uber_reviews_taggedOriginal.csv')
print(f"\n========TAGGED ORIGINAL========")
print(tagged_orignal_df.head())
tagged_boosted_df = pd.read_csv('../data/uber_reviews_taggedBoosted.csv')
print(f"\n========TAGGED BOOSTED========")
print(tagged_boosted_df.head())


========TAGGED ORIGINAL========
                                              review  rating  word_count  \
0      their have many problem but also best service       5           8   
1           it's excellent i loved it thank you uber       5           8   
2    it does the job as it should be, in a nice way!       5          12   
3  i support my family members with the help of uber       5          10   
4         it's good bt it is only.for 1 man or woman       5          10   

   tagged feature_request bug_report   aspect aspect_sentiment  
0       1              No         No  Service         Positive  
1       1              No         No  General         Positive  
2       1              No         No  General         Positive  
3       1              No         No  General         Positive  
4       1             Yes         No  General         Positive  

========TAGGED BOOSTED========
                                              review  word_count  rating  \
0  ""*the wo

In [27]:
print("Number of duplicate rows was tagged_orignal_df: 0")
print("Number of duplicate rows was tagged_boosted_df: 3\n(These were removed in the next cell)\n\nCurrent duplicates: ")


duplicates = tagged_orignal_df.duplicated()
print(f"Number of duplicate rows in tagged_orignal_df: {duplicates.sum()}")

duplicates = tagged_boosted_df.duplicated()
print(f"Number of duplicate rows in tagged_boosted_df: {duplicates.sum()}")





Number of duplicate rows was tagged_orignal_df: 0
Number of duplicate rows was tagged_boosted_df: 3
(These were removed in the next cell)

Current duplicates: 
Number of duplicate rows in tagged_orignal_df: 0
Number of duplicate rows in tagged_boosted_df: 0


In [20]:
tagged_orignal_df = tagged_orignal_df.drop_duplicates()
tagged_boosted_df = tagged_boosted_df.drop_duplicates()

In [34]:
print("Number of exact duplicates in tagged_boosted_df:", tagged_boosted_df[tagged_boosted_df.duplicated(subset=['review'])].shape[0])
print("Number of exact duplicates in tagged_orignal_df:", tagged_orignal_df[tagged_orignal_df.duplicated(subset=['review'])].shape[0])

tagged_boosted_df = tagged_boosted_df.drop_duplicates(subset=['review']).reset_index(drop=True)

Number of exact duplicates in tagged_boosted_df: 0
Number of exact duplicates in tagged_orignal_df: 0


In [35]:
# How many unique reviews remain if you were to drop exact dupes
print("Unique reviews:", tagged_boosted_df['review'].nunique())

# How many duplicate pairs exist (not rows, but pairs)
exact_dupe_rows = tagged_boosted_df[tagged_boosted_df.duplicated(subset=['review'], keep=False)]
print("Rows involved in exact duplication:", exact_dupe_rows.shape[0])
print("Distinct reviews that are duplicated:", tagged_boosted_df.duplicated(subset=['review']).sum())

Unique reviews: 4989
Rows involved in exact duplication: 0
Distinct reviews that are duplicated: 0


In [37]:
print("Boosted sample:")
print(tagged_boosted_df['review'].head(10))
print("\nOriginal sample:")
print(tagged_orignal_df['review'].head(10))

Boosted sample:
0    ""*the worst customer care and worst transport...
1                  guy was excellent give him q raise!
2    ""poor service provider company, i have an err...
3    ""this app did not let me schedule a ride for ...
4    ""worst app.always high prices and drivers alw...
5     ""ac not worked, driver call the whole journey""
6                                  i think ola is best
7    ""when it comes to rides, they're great, but w...
8    am using note 3 samsung. every time this app n...
9    حرامية زفت ضيع وقتى واخد الطريق الطويل عشان يض...
Name: review, dtype: object

Original sample:
0        their have many problem but also best service
1             it's excellent i loved it thank you uber
2      it does the job as it should be, in a nice way!
3    i support my family members with the help of uber
4           it's good bt it is only.for 1 man or woman
5                   easy to use and much more accurate
6         good and comfortable drive also save for us.
7  

In [ ]:
print("Boosted - leading quote artifacts:", tagged_boosted_df[tagged_boosted_df['review'].str.startswith('"')].shape[0])
print("Original - leading quote artifacts:", tagged_orignal_df[tagged_orignal_df['review'].str.startswith('"')].shape[0])

Boosted - leading quote artifacts: 29
Original - leading quote artifacts: 4
Boosted - doubled internal quotes: 29
Original - doubled internal quotes: 0


In [42]:
print("Boosted - leading quote artifacts:", tagged_boosted_df[tagged_boosted_df['review'].str.startswith('"')].shape[0])
print("Original - leading quote artifacts:", tagged_orignal_df[tagged_orignal_df['review'].str.startswith('"')].shape[0])

import re

def clean_quote_artifacts(text):
    if not isinstance(text, str):
        return text
    # Strip any number of leading/trailing quote chars and whitespace
    text = text.strip('" \t')
    # Collapse runs of 2+ quotes down to a single quote (for internal ones like """""""")
    text = re.sub(r'"{2,}', '"', text)
    # Clean up any quotes now left dangling at edges after internal collapse
    text = text.strip('" ')
    return text

tagged_boosted_df['review'] = tagged_boosted_df['review'].apply(clean_quote_artifacts)
tagged_orignal_df['review'] = tagged_orignal_df['review'].apply(clean_quote_artifacts)

print("Boosted - doubled internal quotes:", tagged_boosted_df[tagged_boosted_df['review'].str.contains('"{2,}', regex=True)].shape[0])
print("Original - doubled internal quotes:", tagged_orignal_df[tagged_orignal_df['review'].str.contains('"{2,}', regex=True)].shape[0])

Boosted - leading quote artifacts: 29
Original - leading quote artifacts: 4
Boosted - doubled internal quotes: 0
Original - doubled internal quotes: 0


In [48]:
print(f"Boosted:  {len(tagged_boosted_df):,} rows")
print(f"Original: {len(tagged_orignal_df):,} rows")
print("Boosted nulls:\n", tagged_boosted_df.isnull().sum())
print("\nOriginal nulls:\n", tagged_orignal_df.isnull().sum())
print("Boosted - empty reviews:", tagged_boosted_df['review'].str.strip().eq('').sum())
print("Original - empty reviews:", tagged_orignal_df['review'].str.strip().eq('').sum())
print("Boosted - leading quote artifacts:", tagged_boosted_df[tagged_boosted_df['review'].str.startswith('"')].shape[0])
print("Original - leading quote artifacts:", tagged_orignal_df[tagged_orignal_df['review'].str.startswith('"')].shape[0])
print("Boosted - doubled internal quotes:", tagged_boosted_df[tagged_boosted_df['review'].str.contains('"{2,}', regex=True)].shape[0])
print("Original - doubled internal quotes:", tagged_orignal_df[tagged_orignal_df['review'].str.contains('"{2,}', regex=True)].shape[0])
expected_cols = ['review', 'word_count', 'rating', 'tagged', 'feature_request', 'bug_report', 'aspect', 'aspect_sentiment']
print("Boosted columns match:", list(tagged_boosted_df.columns) == expected_cols)
print("Original columns match:", list(tagged_orignal_df.columns) == expected_cols)
print(tagged_boosted_df[tagged_boosted_df['review'].str.contains('worst customer care', na=False)]['review'].values)

Boosted:  4,989 rows
Original: 4,999 rows
Boosted nulls:
 review               0
word_count           0
rating              84
tagged               0
feature_request      0
bug_report           0
aspect               0
aspect_sentiment     0
dtype: int64

Original nulls:
 review              0
rating              0
word_count          0
tagged              0
feature_request     0
bug_report          0
aspect              0
aspect_sentiment    0
dtype: int64
Boosted - empty reviews: 0
Original - empty reviews: 0
Boosted - leading quote artifacts: 0
Original - leading quote artifacts: 0
Boosted - doubled internal quotes: 0
Original - doubled internal quotes: 0
Boosted columns match: True
Original columns match: False
["*the worst customer care and worst transportation app in sri lanka because of it's staff service. *uber service in sri lanka is a shame to uber technologies, inc. *i expected better service because it's international service but in sri lanka it's better to use local transp

In [44]:
# Are the 84 null ratings clustered in a particular aspect/sentiment or spread across?
print(tagged_boosted_df[tagged_boosted_df['rating'].isnull()]['aspect_sentiment'].value_counts())
print(tagged_boosted_df[tagged_boosted_df['rating'].isnull()]['aspect'].value_counts())

aspect_sentiment
Negative    75
Positive     9
Name: count, dtype: int64
aspect
Driver     23
App        21
Service    20
Payment    12
Pricing     7
General     1
Name: count, dtype: int64


In [ ]:
tagged_boosted_df.to_csv('tagged_boosted_cleaned.csv', index=False)
tagged_orignal_df.to_csv('tagged_original_cleaned.csv', index=False)
# I can confirm, the saved files have no malformed rows/number of columns is correct
print(f"Boosted saved:  {len(tagged_boosted_df):,} rows")
print(f"Original saved: {len(tagged_orignal_df):,} rows")

Boosted saved:  4,989 rows
Original saved: 4,999 rows


In [55]:
# Now I need to convert/map yes / no to integers, same for all  tasks.
# How did everything run earlier? there was a spelling mistake but everything looked fine
tagged_original_df = pd.read_csv('../data/tagged_original_cleaned.csv')
tagged_boosted_df = pd.read_csv('../data/tagged_boosted_cleaned.csv')
# mappings
binary_map = {'Yes': 1, 'No': 0}
aspect_map = {'App': 0, 'Driver': 1, 'General': 2, 'Payment': 3, 'Pricing': 4, 'Service': 5}
sentiment_map = {'Positive': 0, 'Neutral': 1, 'Negative': 2}

tagged_original_df = pd.read_csv('../data/tagged_original_cleaned.csv')
tagged_boosted_df  = pd.read_csv('../data/tagged_boosted_cleaned.csv')

for df in [tagged_boosted_df, tagged_original_df]:
    df['bug_report']       = df['bug_report'].map(binary_map)
    df['feature_request']  = df['feature_request'].map(binary_map)
    df['aspect']           = df['aspect'].map(aspect_map)
    df['aspect_sentiment'] = df['aspect_sentiment'].map(sentiment_map)
    
for df, name in [(tagged_boosted_df, 'Boosted'), (tagged_original_df, 'Original')]:

    # Verification after mapping
    print(f"\n=========== {name} ===========")
    print(df[['bug_report', 'feature_request', 'aspect', 'aspect_sentiment']].isnull().sum())
    print(df[['bug_report', 'feature_request', 'aspect', 'aspect_sentiment']].dtypes)
    print(df[['bug_report', 'feature_request', 'aspect', 'aspect_sentiment']].head(3))





=========== Boosted ===========
bug_report          0
feature_request     0
aspect              0
aspect_sentiment    0
dtype: int64
bug_report          int64
feature_request     int64
aspect              int64
aspect_sentiment    int64
dtype: object
   bug_report  feature_request  aspect  aspect_sentiment
0           0                0       5                 2
1           0                0       1                 0
2           1                0       0                 2

=========== Original ===========
bug_report          0
feature_request     0
aspect              0
aspect_sentiment    0
dtype: int64
bug_report          int64
feature_request     int64
aspect              int64
aspect_sentiment    int64
dtype: object
   bug_report  feature_request  aspect  aspect_sentiment
0           0                0       5                 0
1           0                0       2                 0
2           0                0       2                 0


In [57]:
tagged_boosted_df.to_csv('../data/tagged_boosted_cleaned.csv', index=False)
tagged_original_df.to_csv('../data/tagged_original_cleaned.csv', index=False)

print(f"Boosted saved:  {len(tagged_boosted_df):,} rows")
print(f"Original saved: {len(tagged_original_df):,} rows")

Boosted saved:  4,989 rows
Original saved: 4,999 rows


In [ ]:
print(tagged_boosted_df['bug_report'].unique())
print(tagged_original_df['bug_report'].unique())

# CSV structure checks 

[0 1]
[0 1]

../data/tagged_boosted_cleaned.csv
  Expected columns: 8
  Malformed rows:   0

../data/tagged_original_cleaned.csv
  Expected columns: 8
  Malformed rows:   0


In [ ]:
# Now I can finally split into train/test sets
from sklearn.model_selection import train_test_split

In [ ]:
def split(df, name, seed=67):
    train, temp = train_test_split(df, test_size=0.30, random_state=seed, stratify=df['aspect_sentiment'])
    val, test = train_test_split(temp, test_size=0.50, random_state=seed, stratify=temp['aspect_sentiment'])

    print(f"Train: {len(train):,} ({len(train)/len(df)})")
    print(f"Val:   {len(val):,}   ({len(val)/len(df)})")
    print(f"Test:  {len(test):,}  ({len(test)/len(df)})")
    return train, val, test

tagged_boosted_df = pd.read_csv('../data/tagged_boosted_cleaned.csv')
tagged_original_df = pd.read_csv('../data/tagged_original_cleaned.csv')

train_b, val_b, test_b = split(tagged_boosted_df, 'Boosted')
train_o, val_o, test_o = split(tagged_original_df, 'Original')


Train: 3,492 (0.6999398677089597)
Val:   748   (0.149929845660453)
Test:  749  (0.15013028663058728)
Train: 3,499 (0.6999399879975995)
Val:   750   (0.15003000600120023)
Test:  750  (0.15003000600120023)


In [68]:
splits = [
    (train_b, 'Boosted Train'), (val_b, 'Boosted Val'), (test_b, 'Boosted Test'),
    (train_o, 'Original Train'), (val_o, 'Original Val'), (test_o, 'Original Test')
]

for df, name in splits:
    print(f"\n=========== {name} ===========")
    print("bug_report:\n", df['bug_report'].value_counts(normalize=True))
    print("feature_request:\n", df['feature_request'].value_counts(normalize=True))
    print("aspect:\n", df['aspect'].value_counts(normalize=True))
    print("aspect_sentiment:\n", df['aspect_sentiment'].value_counts(normalize=True))


=========== Boosted Train ===========
bug_report:
 bug_report
0    0.648053
1    0.351947
Name: proportion, dtype: float64
feature_request:
 feature_request
0    0.909507
1    0.090493
Name: proportion, dtype: float64
aspect:
 aspect
0    0.285223
1    0.251432
5    0.154066
2    0.118557
3    0.108820
4    0.081901
Name: proportion, dtype: float64
aspect_sentiment:
 aspect_sentiment
2    0.808706
0    0.182703
1    0.008591
Name: proportion, dtype: float64

=========== Boosted Val ===========
bug_report:
 bug_report
0    0.649733
1    0.350267
Name: proportion, dtype: float64
feature_request:
 feature_request
0    0.895722
1    0.104278
Name: proportion, dtype: float64
aspect:
 aspect
0    0.283422
1    0.236631
5    0.168449
2    0.112299
3    0.108289
4    0.090909
Name: proportion, dtype: float64
aspect_sentiment:
 aspect_sentiment
2    0.808824
0    0.183155
1    0.008021
Name: proportion, dtype: float64

=========== Boosted Test ===========
bug_report:
 bug_report
0    0.670227


In [69]:
def split(df, name, seed=67):
    composite = df['bug_report'].astype(str) + "_" + df['feature_request'].astype(str) + "_" + df['aspect_sentiment'].astype(str)
    train, temp = train_test_split(df, test_size=0.30, random_state=seed, stratify=composite)
    
    composite_temp = temp['bug_report'].astype(str) + "_" + temp['feature_request'].astype(str) + "_" + temp['aspect_sentiment'].astype(str)
    val, test = train_test_split(temp, test_size=0.50, random_state=seed, stratify=composite_temp)

    print(f"Train: {len(train):,} ({len(train)/len(df)})")
    print(f"Val:   {len(val):,}   ({len(val)/len(df)})")
    print(f"Test:  {len(test):,}  ({len(test)/len(df)})")
    return train, val, test

tagged_boosted_df = pd.read_csv('../data/tagged_boosted_cleaned.csv')
tagged_original_df = pd.read_csv('../data/tagged_original_cleaned.csv')

train_b, val_b, test_b = split(tagged_boosted_df, 'Boosted')
train_o, val_o, test_o = split(tagged_original_df, 'Original')

Train: 3,492 (0.6999398677089597)
Val:   748   (0.149929845660453)
Test:  749  (0.15013028663058728)
Train: 3,499 (0.6999399879975995)
Val:   750   (0.15003000600120023)
Test:  750  (0.15003000600120023)


In [71]:
# Using composite key

splits = [
    (train_b, 'Boosted Train'), (val_b, 'Boosted Val'), (test_b, 'Boosted Test'),
    (train_o, 'Original Train'), (val_o, 'Original Val'), (test_o, 'Original Test')
]

for df, name in splits:
    print(f"\n=========== {name} ===========")
    print("bug_report:\n", df['bug_report'].value_counts(normalize=True))
    print("feature_request:\n", df['feature_request'].value_counts(normalize=True))
    print("aspect:\n", df['aspect'].value_counts(normalize=True))
    print("aspect_sentiment:\n", df['aspect_sentiment'].value_counts(normalize=True))
    df.to_csv(f'{name.lower().replace(" ", "_")}.csv', index=False)    # 6 seperate files are saved


=========== Boosted Train ===========
bug_report:
 bug_report
0    0.651489
1    0.348511
Name: proportion, dtype: float64
feature_request:
 feature_request
0    0.905212
1    0.094788
Name: proportion, dtype: float64
aspect:
 aspect
0    0.280928
1    0.250286
5    0.152348
2    0.118270
3    0.114261
4    0.083906
Name: proportion, dtype: float64
aspect_sentiment:
 aspect_sentiment
2    0.808706
0    0.182990
1    0.008305
Name: proportion, dtype: float64

=========== Boosted Val ===========
bug_report:
 bug_report
0    0.65107
1    0.34893
Name: proportion, dtype: float64
feature_request:
 feature_request
0    0.906417
1    0.093583
Name: proportion, dtype: float64
aspect:
 aspect
0    0.286096
1    0.251337
5    0.159091
2    0.113636
3    0.097594
4    0.092246
Name: proportion, dtype: float64
aspect_sentiment:
 aspect_sentiment
2    0.807487
0    0.183155
1    0.009358
Name: proportion, dtype: float64

=========== Boosted Test ===========
bug_report:
 bug_report
0    0.65287
1  

In [72]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

for df, name in [(tagged_boosted_df, 'Boosted'), (tagged_original_df, 'Original')]:
    lengths = df['review'].apply(lambda x: len(tokenizer.encode(x)))
    print(f"\n{name}")
    print(f"  Max tokens:    {lengths.max()}")
    print(f"  Mean tokens:   {lengths.mean():.1f}")
    print(f"  Over 128:      {(lengths > 128).sum()} ({(lengths > 128).mean():.1%})")
    print(f"  Over 256:      {(lengths > 256).sum()} ({(lengths > 256).mean():.1%})")

PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]


Boosted
  Max tokens:    471
  Mean tokens:   49.8
  Over 128:      180 (3.6%)
  Over 256:      21 (0.4%)

Original
  Max tokens:    327
  Mean tokens:   29.3
  Over 128:      58 (1.2%)
  Over 256:      5 (0.1%)


In [ ]:
# Let's also see if the longer reviews are more likely to be bug reports, feature requests, or have certain aspects/sentiments
# This finding allows us to make a better decision on the max_length for the model, which will increase the quality of the model,
# the time to train will be longer but it is not worth removing valuable information from longer reviews which are mostly bug reports and feature requests 
# with negative sentiment (the most important ones to classify correctly)
for df, name in [(tagged_boosted_df, 'Boosted'), (tagged_original_df, 'Original')]:
    lengths = df['review'].apply(lambda x: len(tokenizer.encode(x)))
    mask_128 = lengths > 128
    mask_256 = lengths > 256
    
    print(f"\n{name}")
    print(f"  Max tokens:    {lengths.max()}")
    print(f"  Mean tokens:   {lengths.mean():.1f}")
    
    for mask, label in [(mask_128, 'Over 128'), (mask_256, 'Over 256')]:
        truncated = df[mask]
        print(f"\n  {label}: {mask.sum()} ({mask.mean():.1%})")
        print(f"    bug_report=1:      {truncated['bug_report'].sum()} ({truncated['bug_report'].mean():.1%} of truncated)")
        print(f"    feature_request=1: {truncated['feature_request'].sum()} ({truncated['feature_request'].mean():.1%} of truncated)")
        print(f"    aspect counts:     {truncated['aspect'].value_counts().to_dict()}")
        print(f"    sentiment counts:  {truncated['aspect_sentiment'].value_counts().to_dict()}")


Boosted
  Max tokens:    471
  Mean tokens:   49.8

  Over 128: 180 (3.6%)
    bug_report=1:      68 (37.8% of truncated)
    feature_request=1: 17 (9.4% of truncated)
    aspect counts:     {1: 57, 0: 47, 5: 36, 3: 26, 4: 12, 2: 2}
    sentiment counts:  {2: 165, 0: 15}

  Over 256: 21 (0.4%)
    bug_report=1:      6 (28.6% of truncated)
    feature_request=1: 1 (4.8% of truncated)
    aspect counts:     {1: 6, 5: 6, 3: 5, 0: 4}
    sentiment counts:  {2: 21}

Original
  Max tokens:    327
  Mean tokens:   29.3

  Over 128: 58 (1.2%)
    bug_report=1:      24 (41.4% of truncated)
    feature_request=1: 29 (50.0% of truncated)
    aspect counts:     {0: 25, 5: 13, 1: 8, 2: 5, 4: 4, 3: 3}
    sentiment counts:  {2: 45, 0: 10, 1: 3}

  Over 256: 5 (0.1%)
    bug_report=1:      2 (40.0% of truncated)
    feature_request=1: 2 (40.0% of truncated)
    aspect counts:     {0: 2, 4: 1, 1: 1, 5: 1}
    sentiment counts:  {2: 5}
